In [2]:
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler


PRIVATE = True

# openai 결제를 하지 않아서 ollama를 사용하였습니다.
model = "mistral:latest" if PRIVATE else "gpt-3.5-turbo"
ChatModel = ChatOllama if PRIVATE else ChatOpenAI

chat = ChatModel(
    model=model,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [6]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
    {
        "movie": "Avatar",
        "answer": """
director: James Cameron
main cast: Sam Worthington, Zoe Saldana, Stephen Lang, Michelle Rodriguez, Sigourney Weaver
budget: 237000000
box office revenue: 2923706026
genres: Action,Adventure,Fantasy,Science Fiction
synopsis: In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.
""",
    },
    {
        "movie": "Avengers: Endgame",
        "answer": """
director: Anthony Russo
main cast: Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson
budget: 356000000
box office revenue: 2799439100
genres: Adventure,Science Fiction,Action
synopsis: After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos' actions and restore order to the universe once and for all, no matter what consequences may be in store.
""",
    },
    {
        "movie": "Titanic",
        "answer": """
director: James Cameron
main cast: Leonardo DiCaprio, Kate Winslet, Billy Zane, Kathy Bates
budget: 200000000
box office revenue: 2264162353
genres: Drama,Romance
synopsis: 101-year-old Rose DeWitt Bukater tells the story of her life aboard the Titanic, 84 years later. A young Rose boards the ship with her mother and fiancé. Meanwhile, Jack Dawson and Fabrizio De Rossi win third-class tickets aboard the ship. Rose tells the whole story from Titanic's departure through to its death—on its first and last voyage—on April 15, 1912.
""",
    },
    {
        "movie": "Frozen",
        "answer": """
director: Chris Buck
main cast: Kristen Bell, Idina Menzel, Jonathan Groff
budget: 150000000
box office revenue: 1274219009
genres: Animation,Family,Adventure,Fantasy
synopsis: Young princess Anna of Arendelle dreams about finding true love at her sister Elsa’s coronation. Fate takes her on a dangerous journey in an attempt to end the eternal winter that has fallen over the kingdom. She's accompanied by ice delivery man Kristoff, his reindeer Sven, and snowman Olaf. On an adventure where she will find out what friendship, courage, family, and true love really means.
""",
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know {movie}?"),
        ("ai", "{answer}"),
    ]
)


example_template = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

full_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a movie expert. you only give short answers.",
        ),
        example_template,
        ("human", "What do you know {movie}?"),
    ]
)


chain = full_prompt | chat

chain.invoke({"movie": "Iron Man"})

 director: Jon Favreau
main cast: Robert Downey Jr., Terrence Howard, Jeff Bridges, Gwyneth Paltrow
budget: 140000000
box office revenue: 585370619
genres: Action,Adventure,Science Fiction
synopsis: After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.

AIMessage(content=' director: Jon Favreau\nmain cast: Robert Downey Jr., Terrence Howard, Jeff Bridges, Gwyneth Paltrow\nbudget: 140000000\nbox office revenue: 585370619\ngenres: Action,Adventure,Science Fiction\nsynopsis: After being held captive in an Afghan cave, billionaire engineer Tony Stark creates a unique weaponized suit of armor to fight evil.')